In [18]:
import os
import pandas as pd
import ast
import pickle
import queue
import networkx as nx
from utils.ProjectPath import ProjectPath
import copy
path = ProjectPath()
chain_stats_path = os.path.join(path.panv2_scammer_chain_path, "chain_stats.csv")

df = pd.read_csv(chain_stats_path)

ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX


In [19]:
df.describe()

,chain_length,num_scams_avg,trans_amt_avg,trans_time_diff_avg
count,14028.000000,14028.000000,14028.000000,5.583000e+03
mean,3.707870,1.821315,13.252944,1.545478e+05
std,8.304718,3.899602,52.287283,7.070301e+05
min,2.000000,0.500000,0.001572,8.700000e+01
25%,2.000000,1.000000,1.057682,3.597000e+03
50%,2.000000,1.000000,2.309997,2.031000e+04
75%,3.000000,1.500000,6.226213,8.749050e+04
max,713.000000,142.250000,1485.531661,2.175045e+07


In [20]:
chain_stats_path = os.path.join(path.panv2_star_shape_path, "star_stats.csv")
star_df = pd.read_csv(chain_stats_path)
star_df['scam_duration'] = star_df['scam_duration'] / (3600 * 24)
star_df

,star_type,center_address,star_size,funds_in_to_center_avg,funds_in_to_center_total,funds_out_from_center_avg,funds_out_from_center_total,scam_duration,num_scams_total
0,IN,0x9ab3913cabd4ae9a742ecc18deef6dc703d9c589,5,2.815647,14.078233,NaN,NaN,83.246262,5
1,IN,0x735826b061c7d64a8be006f340a6fa753d1fa9a7,9,0.892922,8.036297,NaN,NaN,114.920185,10
2,IN,0xd11a88c4b4fbd7d6ce37e842417a8f5443c9c799,26,0.371099,9.648587,NaN,NaN,1.122731,29
3,IN,0xd21f36a8d1cda0cd31e45e2dc2defdd86150a607,26,1.677801,43.622837,NaN,NaN,212.265382,32
4,IN,0xd0411fb6fab9a1328f74d9adfc96f53be3532fb3,388,1.340826,520.240676,NaN,NaN,281.846343,572
...,...,...,...,...,...,...,...,...,...
2361,IN_OUT,0xc913c92f6aa3f58ff25c6a459815989f0c92e3ec,6,1.315615,7.893688,0.900000,5.400000,0.530613,15
2362,IN_OUT,0x1c4a936c1c11bdbaf4a0258bc05e5cdcd2b3491e,6,2.901158,17.406947,2.890000,17.340000,15.138056,6
2363,IN_OUT,0x08d4615245cab28fa5447b9c4f1c821b59885aee,5,17.007374,85.036870,17.000000,85.000000,38.467917,5
2364,IN_OUT,0x74428f99d8db226f2430c0cc68360476fd95f33d,5,0.528324,2.641620,0.380000,1.900000,2.153137,5


In [21]:
star_df = star_df[['star_type', 'star_size', 'funds_in_to_center_total', 'funds_out_from_center_total', 'scam_duration', 'num_scams_total']]
star_df_groups = star_df.groupby("star_type")
star_df_groups.describe(percentiles=[])

star_size                                         \
              count       mean        std  min  50%    max   
star_type                                                    
IN           1815.0  21.398347  40.589027  5.0  9.0  505.0   
IN_OUT        398.0  15.550251  20.908129  5.0  9.0  200.0   
OUT           153.0  11.849673  14.754469  5.0  7.0  129.0   

          funds_in_to_center_total                                  ...  \
                             count       mean        std       min  ...   
star_type                                                           ...   
IN                          1815.0  97.983157  391.40940  0.536278  ...   
IN_OUT                       398.0  75.670718  222.87684  0.331398  ...   
OUT                            0.0        NaN        NaN       NaN  ...   

          scam_duration                                  num_scams_total  \
                    std       min        50%         max           count   
star_type                                                                  
IN           157.525283  0.000451  65.649861  915.702778          1815.0   
IN_OUT        74.675292  0.017014   9.135694  857.207581           398.0   
OUT           81.067290  0.000590  20.097141  448.739213           153.0   

                                                   
                mean        std  min   50%    max  
star_type                                          
IN         28.583471  55.874145  5.0  12.0  843.0  
IN_OUT     20.379397  30.002675  5.0  10.0  222.0  
OUT        17.908497  51.344215  5.0   7.0  616.0  

[3 rows x 30 columns]

In [22]:
#Chain
def extract_scammers(chain, col):
    scammers = set()
    for idx, row in chain.iterrows():
        chain_ast = ast.literal_eval(row.iloc[col])
        for n in chain_ast:
            scammers.add(n[0])
    return scammers


uni_simple_chain = pd.read_csv("data/patterns/chain/uniswap_simple_chain.csv", delimiter="|")
pan_simple_chain = pd.read_csv("data/patterns/chain/pancakeswap_simple_chain.csv", delimiter="|")
uni_chain_scammers = extract_scammers(uni_simple_chain, 1)
pan_chain_scammers = extract_scammers(pan_simple_chain, 1)
print(len(uni_chain_scammers))
print(len(pan_chain_scammers))

19977
1245


In [23]:
#Star
def extract_scammers_from_star(star):
    scammers = extract_scammers(star, 2)
    scammers.update(set(star.center_address.values.tolist()))
    # print(len(scammers))
    return scammers


uni_in_star = pd.read_csv("data/patterns/star/uniswap_in_stars.csv", delimiter="|")
uni_out_star = pd.read_csv("data/patterns/star/uniswap_out_stars.csv", delimiter="|")
uni_in_out_star = pd.read_csv("data/patterns/star/uniswap_in_out_stars.csv", delimiter="|")
pan_in_star = pd.read_csv("data/patterns/star/pancakeswap_in_stars.csv", delimiter="|")
pan_out_star = pd.read_csv("data/patterns/star/pancakeswap_out_stars.csv", delimiter="|")
pan_in_out_star = pd.read_csv("data/patterns/star/pancakeswap_in_out_stars.csv", delimiter="|")

uni_in_star_scammers = extract_scammers_from_star(uni_in_star)
uni_out_star_scammers = extract_scammers_from_star(uni_out_star)
uni_in_out_star_scammers = extract_scammers_from_star(uni_in_out_star)
uni_star_scammers = uni_in_star_scammers | uni_out_star_scammers | uni_in_out_star_scammers

pan_in_star_scammers = extract_scammers_from_star(pan_in_star)
pan_out_star_scammers = extract_scammers_from_star(pan_out_star)
pan_in_out_star_scammers = extract_scammers_from_star(pan_in_out_star)
pan_star_scammers = pan_in_star_scammers | pan_out_star_scammers | pan_in_out_star_scammers

print(len(uni_star_scammers))
print(len(pan_star_scammers))


32597
1428


In [24]:
# Majority Flow
all_funding_txs = set()
all_funding_tx_hashes = set()
all_scammer_addrs = set()
visited_tx = set()
atomic_MSF_groups = []
F_txs = {}
B_txs = {}

class MaximalScamFundingCluster():
    def __init__(self, tx_id):
        self.id = tx_id.hash
        self.V = set()
        self.E = set()
        self.inputs = set()
        self.outputs = set()
        q = queue.Queue()
        q.put(tx_id)
        # only valid funding txs in queue
        while not q.empty():
            tx = q.get()
            self.E.add(tx)
            sender = tx.sender
            receiver = tx.to
            visited_tx.add(tx.hash)
            self.V.add(sender)
            self.V.add(receiver)
            self.inputs.add(sender)
            self.outputs.add(receiver)

            if all(b_tx.hash in all_funding_tx_hashes for b_tx in B_txs[sender]):
                for b_tx in B_txs[sender]:
                    if b_tx.hash in all_funding_tx_hashes and b_tx.hash not in visited_tx:
                        q.put(b_tx)
            if all(f_tx.hash in all_funding_tx_hashes for f_tx in F_txs[receiver]):
                for f_tx in F_txs[receiver]:
                    if f_tx.hash in all_funding_tx_hashes and f_tx.hash not in visited_tx:
                        q.put(f_tx)

    def merge(self, other_group):
        E_hashes = set()
        E_ = set()
        for e in self.E.union(other_group.E):
            if e.hash not in E_hashes:
                E_hashes.add(e.hash)
                E_.add(e)
        self.E = copy.deepcopy(E_)

        self.inputs = self.inputs.union(other_group.inputs) - self.V.intersection(other_group.V)
        self.outputs = self.outputs.union(other_group.outputs) - self.V.intersection(other_group.V)

        self.V = self.V.union(other_group.V)
MF_uni_scammers = set()   
MF_uni_cluster = []
with open(f'data/patterns/majority_flow/MSF_clusters_univ2.pkl', 'rb') as file:
    connected_components, MSF_clusters = pickle.load(file)
    max = 0
    for i, cluster in enumerate(MSF_clusters):
      MF_uni_scammers.update(cluster.V)
      MF_uni_cluster.append(cluster.V)
print(len(MF_uni_scammers))


MF_pan_scammers = set()      
with open(f'data/patterns/majority_flow/MSF_clusters_panv2.pkl', 'rb') as file:
    connected_components, MSF_clusters = pickle.load(file)
    max = 0
    for i, cluster in enumerate(MSF_clusters):
      MF_pan_scammers.update(cluster.V)
print(len(MF_pan_scammers))



18692
1903


In [25]:
uni_total_scammer = 145654
pan_total_scammer = 18346
uni_total_pattern_scammer =  MF_uni_scammers | uni_star_scammers | uni_chain_scammers
pan_total_pattern_scammer =  MF_pan_scammers | pan_star_scammers | pan_chain_scammers
print(len(uni_total_pattern_scammer))
print(len(pan_total_pattern_scammer))


54054
3369


In [26]:
print("UNISWAP")
print("Total unique scammers:", uni_total_scammer)
print("Simple chain scammers:", len(uni_chain_scammers), f"({ len(uni_chain_scammers) / uni_total_scammer * 100} %)")
print("Star scammers:", len(uni_star_scammers), f"({ len(uni_star_scammers) / uni_total_scammer * 100} %)")
print("MSF scammers:", len(MF_uni_scammers), f"({ len(MF_uni_scammers) / uni_total_scammer * 100} %)")
print("All pattern scammers:", len(uni_total_pattern_scammer), f"({ len(uni_total_pattern_scammer) / uni_total_scammer * 100} %)")

print("PANCAKESWAP")
print("Total unique scammers:", pan_total_scammer)
print("Simple chain scammers:", len(pan_chain_scammers), f"({ len(pan_chain_scammers) / pan_total_scammer * 100} %)")
print("Star scammers:", len(pan_star_scammers), f"({ len(pan_star_scammers) / pan_total_scammer * 100} %)")
print("MSF scammers:", len(MF_pan_scammers), f"({ len(MF_pan_scammers) / pan_total_scammer * 100} %)")
print("All pattern scammers:", len(pan_total_pattern_scammer), f"({ len(pan_total_pattern_scammer) / pan_total_scammer * 100} %)")


UNISWAP
Total unique scammers: 145654
Simple chain scammers: 19977 (13.715380284784489 %)
Star scammers: 32597 (22.37974926881514 %)
MSF scammers: 18692 (12.8331525395801 %)
All pattern scammers: 54054 (37.11123621733698 %)
PANCAKESWAP
Total unique scammers: 18346
Simple chain scammers: 1245 (6.786220429521421 %)
Star scammers: 1428 (7.783713070969149 %)
MSF scammers: 1903 (10.372833315164067 %)
All pattern scammers: 3369 (18.36367600566881 %)


In [29]:
for idx in range(len(MF_uni_cluster)):
    cluster = MF_uni_cluster[idx]
    print("Cluster:", idx)
    for address in cluster:
        print("\t"+address)

Cluster: 0
	0x8c9e160d03a98d177b1d10fc863af5b6e37815cf
	0xcb1b77842a9c6e510605734ecd730efc7a0d0960
	0x86f644c23ec91e84577f9332410d018b01c3c079
	0xac79b36ef29947f254175cb020f28552ade068d3
	0xc352c78e4e0b474f8d87a7b400c5f48966a6016d
	0x0fdd3d1cdad6940f4deafb6d1d03c84562659f53
	0x198f92d371d63cb55f52f32ac4e1aeabafcb70b0
	0x74c4257e9961b803194a4523ccf7eb43f49dc83c
	0x056c04680bd0c4fe125ea612936b27dd59fa5b5a
	0x0c6bef54b3e238bf5ccd9268cdb2482dcf0f4a6f
	0x48b122706b0a0a63c151bb16839e236835932cc9
	0xa7aafd1d834e5a9f4b835d8634eb96a610539cec
	0x900dabc2e45c7b7f1c3736ec3b105b95ca7ccb6c
	0xf20a3e18ff616864fe6ae4191214ccd18c99c5d6
	0xe72b6dcef488b8a9b64d2f8fbbd44fa63c2fb6db
	0x6b4c1edb19117979e80dd9796880726557426cf7
	0x6ad2abc1cbfa4065f81896c1e8ab3c8168f310e6
	0xc8f42863e09614963f2c87a8e563e264caa4735b
Cluster: 1
	0x68fae54be542ac03b4ec1bbf523132a3b7f345d5
	0xf7f3509118957ede246de361f82dc65155452f64
Cluster: 2
	0xc1bec8e3befd1d0860519d5cf1da0c0751dfcec9
	0xb823a2f43aea4555f569cfaa1eefc910987df9a9

In [30]:
new_pan_total_scammer = 238280

new_pan_simple_chain = pd.read_csv("data/patterns/pan_new/simple_chain.csv", delimiter="|")
new_pan_chain_scammers = extract_scammers(new_pan_simple_chain, 1)
new_pan_in_star = pd.read_csv("data/patterns/pan_new/in_stars.csv", delimiter="|")
new_pan_out_star = pd.read_csv("data/patterns/pan_new/out_stars.csv", delimiter="|")
new_pan_in_out_star = pd.read_csv("data/patterns/pan_new/in_out_stars.csv", delimiter="|")
new_pan_in_star_scammers = extract_scammers_from_star(new_pan_in_star)
new_pan_out_star_scammers = extract_scammers_from_star(new_pan_out_star)
new_pan_in_out_star_scammers = extract_scammers_from_star(new_pan_in_out_star)
new_pan_star_scammers = new_pan_in_star_scammers | new_pan_out_star_scammers | new_pan_in_out_star_scammers

new_MF_pan_scammers = set()      
with open(f'data/patterns/pan_new/MSF_clusters_panv2.pkl', 'rb') as file:
    connected_components, MSF_clusters = pickle.load(file)
    max = 0
    for i, cluster in enumerate(MSF_clusters):
      new_MF_pan_scammers.update(cluster.V)

new_pan_total_pattern_scammer =  new_MF_pan_scammers | new_pan_star_scammers | new_pan_chain_scammers
print("NEW PANCAKESWAP")
print("Total unique scammers:", new_pan_total_scammer)
print("Simple chain scammers:", len(new_pan_chain_scammers), f"({ len(new_pan_chain_scammers) / new_pan_total_scammer * 100} %)")
print("Star scammers:", len(new_pan_star_scammers), f"({ len(new_pan_star_scammers) / new_pan_total_scammer * 100} %)")
print("MSF scammers:", len(new_MF_pan_scammers), f"({ len(new_MF_pan_scammers) / new_pan_total_scammer * 100} %)")
print("All pattern scammers:", len(new_pan_total_pattern_scammer), f"({ len(new_pan_total_pattern_scammer) / new_pan_total_scammer * 100} %)")

NEW PANCAKESWAP
Total unique scammers: 238280
Simple chain scammers: 52014 (21.82894074198422 %)
Star scammers: 48345 (20.28915561524257 %)
MSF scammers: 58309 (24.47079066644284 %)
All pattern scammers: 105660 (44.34278999496391 %)
